In [1]:
from itertools import permutations, combinations

gap('LoadPackage("Cryst")')

true

# Find normalizers for all Point Groups of crystallographic groups in R^2

In [2]:
var('a,b,c,d')

(a, b, c, d)

In [3]:
A = matrix([[a, b], [c, d]])

## There are 11 different Point Groups 

In [4]:
point_groups = set() 
for n in range(1, 18):
    G = gap(f'SpaceGroupIT(2, {n})') 
    S = G.PointGroup() 
    
    point_groups.add(str(S))

point_groups

{'Group([ [ [ -1, 0 ], [ 0, -1 ] ] ])',
 'Group([ [ [ -1, 0 ], [ 0, -1 ] ], [ [ -1, 0 ], [ 0, 1 ] ] ])',
 'Group([ [ [ -1, 0 ], [ 0, -1 ] ], [ [ 0, 1 ], [ -1, 0 ] ] ])',
 'Group([ [ [ -1, 0 ], [ 0, -1 ] ], [ [ 0, 1 ], [ -1, 0 ] ], \n  [ [ -1, 0 ], [ 0, 1 ] ] ])',
 'Group([ [ [ -1, 0 ], [ 0, 1 ] ] ])',
 'Group([ [ [ 0, 1 ], [ -1, -1 ] ] ])',
 'Group([ [ [ 0, 1 ], [ -1, -1 ] ], [ [ -1, 0 ], [ 0, -1 ] ] ])',
 'Group([ [ [ 0, 1 ], [ -1, -1 ] ], [ [ -1, 0 ], [ 0, -1 ] ], \n  [ [ 0, -1 ], [ -1, 0 ] ] ])',
 'Group([ [ [ 0, 1 ], [ -1, -1 ] ], [ [ 0, -1 ], [ -1, 0 ] ] ])',
 'Group([ [ [ 0, 1 ], [ -1, -1 ] ], [ [ 0, 1 ], [ 1, 0 ] ] ])',
 'Group([],[ [ 1, 0 ], [ 0, 1 ] ])'}

In [5]:
def normalize_expressions(exps, allowed=None):
    """Replace all variables except allowed (default: a b c d) with {x0,x1,x2,x3...}"""
    extra_args = set()
                          
    if allowed is None: 
        allowed = var('a b c d')
        
    for exp in exps: 
#         print(exp)
        extra_args |= {el for el in exp.args() if el not in allowed}
        
    res = []
    for exp in exps:
        for i, el in enumerate(extra_args): 
            exp = exp.subs({var(el): var(f'x{i}')})
        res.append(exp)
    return tuple(res)


var('a b c')
var('r1 r2 r3')

exp = [a == r1 + r2 + r3, b == r2, c == r1]
normalize_expressions(exp)

(a == x0 + x1 + x2, b == x1, c == x2)

In [6]:
expression = exp[0]

In [7]:
def prepare_gap_env():
    gap('LoadPackage("Cryst");;Read("../working/el2word.g")')
    
    
def el2word(G, el, verbose=False):
    """Represent, given an element of a finite group G, 
    as the word over generators."""
    
    prepare_gap_env()
    gap.set('G', G)
    gap.set('g', el)
    

    gens_dict = {f'g_{i}' : el for i, el in enumerate(list(G.GeneratorsOfGroup()))}
    
    gens = '"' + '", "'.join(sorted(gens_dict.keys())) + '"'
    expression = f'FreeGroup({gens})'
    if verbose:
        print("Free group to be created:", expression)
        print("Translation of generators:", gens_dict)

    gap.set("F", expression)

    res = gap('ElementAsWordGeneratorsPointGroup(G, g, F)')
    return str(res).split("*"), gens_dict 


def extend_permutation(G, el, gens, perm):        
    """Extend given permutation on generators onto the entire group."""
    word, trans = el2word(G, el)
    
    if el == G.One(): 
        return el 
    
    res = G.One()    
    for g in word: 
        power = 1
        if "^" in g: 
            g, power = g.split('^')
            power = int(power)
        index = gens.index(trans[g])
        g_to = perm[index]
#         index_to = perm[index]
#         g_to = gens[index_to] ** power 
        res *= g_to ** power
    return res


In [8]:
prepare_gap_env()

In [9]:
G = gap.SpaceGroupIT(2, 10).PointGroup()
G

Group([ [ [ -1, 0 ], [ 0, -1 ] ], [ [ 0, 1 ], [ -1, 0 ] ] ])

In [10]:
G.One() ** (-1)

[ [ 1, 0 ], [ 0, 1 ] ]

In [11]:
list(G.AsList())[0].Order()

1

In [12]:
perm = [1, 0]
gens = list(G.GeneratorsOfGroup())
el = list(G.AsList())[1]
extend_permutation(G, el, gens, perm)

TypeError: Gap produced error output
Error, <gens> and <imgs> must be lists of same length

   executing \$sage3:=ElementAsWordGeneratorsPointGroup(G, g, F);;

In [13]:
a, b, = list(G.GeneratorsOfGroup())

In [85]:
a 

[ [ -1, 0 ], [ 0, -1 ] ]

In [28]:
b

[ [ 0, 1 ], [ -1, 0 ] ]

In [12]:
x, trans = el2word(G, a * b, verbose=True)
x

Free group to be created: FreeGroup("g_0", "g_1")
Translation of generators: {'g_0': [ [ -1, 0 ], [ 0, -1 ] ], 'g_1': [ [ 0, 1 ], [ -1, 0 ] ]}


['g_0', 'g_1']

In [11]:
c

[ [ 1, 0, 0 ], [ 0, 1, 0 ], [ 1, 0, 1 ] ]

In [48]:
G.PointGroup()

Group([ [ [ -1, 0 ], [ 0, -1 ] ], [ [ 0, 1 ], [ -1, 0 ] ] ])

In [14]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'


def create_symbolic_matrix(dim, use_alphabet=False):
    if use_alphabet and dim * dim > len(alphabet): 
        raise ValueError(f"Can't use alphabet for matrix {dim}x{dim} due to lack of letters.")
    A = []
    args = []
    for i in range(dim):
        row = []
        for j in range(dim):
            if use_alphabet: 
                row.append(var(alphabet[i * dim + j]))
            else:
                row.append(var(f'a_{i}{j}'))
            args.append(row[-1])
        A.append(row)
    A = matrix(A)
    return A, args


create_symbolic_matrix(5, use_alphabet=True)

(
[a b c d e]                                                                             
[f g h i j]                                                                             
[k l m n o]                                                                             
[p q r s t]                                                                             
[u v w x y], [a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y]
)

In [15]:
def sol2matrix(solution, dim=2, use_alphabet=False):
    if use_alphabet and dim * dim > len(alphabet): 
        raise ValueError(f"Can't use alphabet for matrix {dim}x{dim} due to lack of letters.")
    A = []
    sol_dict = {sol.left(): sol.right() for sol in solution}
    
    for i in range(dim):
        row = []
        for j in range(dim):
            if use_alphabet: 
                row.append(sol_dict[var(alphabet[i * dim + j])])
            else:
                row.append(sol_dict[var(f'a_{i}{j}')])

        A.append(row)
    A = matrix(A)
    return A


def normalizers_v2(n, dim=2, verbose=False, use_alphabet=False, normalize_exp=True, to_matrix=True): 
    G = gap(f'SpaceGroupIT({dim}, {n})')
    S = G.PointGroup() 
    s_elements = S.AsList()
    gens = list(S.GeneratorsOfGroup())
    
    if verbose: 
        print('\n====================================================')
        print(n, 'point group:', S)

        print('group elements:')
        print(*s_elements, sep='\n')
        print('\n----------------normalizers-------------------------')
    
    A, args = create_symbolic_matrix(dim, use_alphabet=use_alphabet)
    
    conditions = [] 
    found_solutions = set() 
#     for perm in permutations(range(len(gens))):
    for perm in permutations(s_elements, len(gens)):
        # validate permutation
        succ = True
        for gen, s_to in zip(gens, perm): 
            if gen.Order() != s_to.Order(): 
                succ = False 
                break
        if not succ: 
            continue
            
        # build conditions AX = YA for every X -> Y due to chosen permutation 
        cond_perm = set()
        for X in s_elements:
            Y = extend_permutation(S, X, gens, perm)
            
            X, Y = matrix(QQ, X), matrix(QQ, Y)
            cond_i = A * X - Y * A
            for el in cond_i: 
                cond_perm = cond_perm.union(set(el))
        
        eq = [cond == 0 for cond in cond_perm]
        for res in solve(eq, *args):
            if not res: 
                if verbose: print(res) 
                continue
                
            res = tuple(res) 
            if normalize_exp: 
                res = normalize_expressions(res, allowed=args)
                
            if res not in found_solutions and verbose: 
                if to_matrix:
                    print(sol2matrix(res, dim=dim, use_alphabet=use_alphabet), end='\n\n')
                else: 
                    print(res)
                
            found_solutions.add(res)
            
    if to_matrix:
        return [sol2matrix(solution, dim=dim, use_alphabet=use_alphabet) for solution in found_solutions]
    else: 
        return found_solutions

In [16]:
normalizers_v2(4, dim=2, verbose=True, use_alphabet=True)


4 point group: Group([ [ [ -1, 0 ], [ 0, 1 ] ] ])
group elements:
[ [ 1, 0 ], [ 0, 1 ] ]
[ [ -1, 0 ], [ 0, 1 ] ]

----------------normalizers-------------------------
[x0  0]
[ 0 x1]



[
[x0  0]
[ 0 x1]
]

In [168]:
res = normalizers_v2(112, dim=3, verbose=True, use_alphabet=True, to_matrix=True)


112 point group: Group([ [ [ -1, 0, 0 ], [ 0, -1, 0 ], [ 0, 0, 1 ] ], 
  [ [ 0, -1, 0 ], [ 1, 0, 0 ], [ 0, 0, -1 ] ], 
  [ [ -1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, -1 ] ] ])
group elements:
[ [ 1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ] ]
[ [ -1, 0, 0 ], [ 0, -1, 0 ], [ 0, 0, 1 ] ]
[ [ 0, 1, 0 ], [ -1, 0, 0 ], [ 0, 0, -1 ] ]
[ [ 0, -1, 0 ], [ 1, 0, 0 ], [ 0, 0, -1 ] ]
[ [ 1, 0, 0 ], [ 0, -1, 0 ], [ 0, 0, -1 ] ]
[ [ -1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, -1 ] ]
[ [ 0, 1, 0 ], [ 1, 0, 0 ], [ 0, 0, 1 ] ]
[ [ 0, -1, 0 ], [ -1, 0, 0 ], [ 0, 0, 1 ] ]

----------------normalizers-------------------------
[ 0 x1  0]
[x1  0  0]
[ 0  0 x0]

[-x0   0   0]
[  0  x0   0]
[  0   0  x1]

[-x0  x0   0]
[ x0  x0   0]
[  0   0   0]

[ x0  x0   0]
[ x0 -x0   0]
[  0   0   0]

[  0 -x0   0]
[ x0   0   0]
[  0   0  x1]

[x0  0  0]
[ 0 x0  0]
[ 0  0 x1]

[ x0  x0   0]
[-x0  x0   0]
[  0   0   0]

[ x0 -x0   0]
[ x0  x0   0]
[  0   0   0]

[0 0 0]
[0 0 0]
[0 0 0]

[ 0  0  0]
[ 0  0  0]
[ 0  0 x0]



In [152]:
i = 10
solutions_v2 = normalizers_v2(i, dim=2, verbose=False, use_alphabet=True, to_matrix=False)
solutions_v1 = normalizers(i, dim=2, verbose=False, use_alphabet=True, to_matrix=False)

for el in solutions_v1: 
    if el not in solutions_v2: 
        print(el)

(a == x1, b == -x0, c == x0, d == x1)


In [113]:
def normalizers(n, dim=2, use_alphabet=False, verbose=False, normalize_exp=True, to_matrix=True):
    G = gap(f'SpaceGroupIT({dim}, {n})')
    S = G.PointGroup()
    s_elements = [matrix(QQ, el) for el in S.AsList()]
    
    if verbose: 
        print('\n====================================================')
        print(n, 'point group:', S)

        print('group elements:')
        print(*s_elements, sep='\n')
        print('\n----------------normalizers-------------------------')
    
    A, args = create_symbolic_matrix(dim=dim, use_alphabet=use_alphabet)
    
    conditions = []
    found_solutions = set()    
    for perm in permutations(range(len(s_elements))):
        
        # check whether permutation maps elements with same order
        check = True
        for i in range(len(s_elements)):
            if S.AsList()[i + 1].Order() != S.AsList()[perm[i] + 1].Order():
                check = False 
                break 
        if not check: 
            continue
        
        # build conditions AX = YA for every X -> Y due to chosen permutation 
        cond_perm = set()
        for i in range(len(s_elements)):
            cond_i = A * s_elements[i] - s_elements[perm[i]] * A
            for el in cond_i: 
                cond_perm = cond_perm.union(set(el))
    
        eq = [cond == 0 for cond in cond_perm]
        for res in solve(eq, *args):
            if not res: 
                if verbose: print(res) 
                continue
            res = tuple(res) 
            if normalize_exp:
                res = normalize_expressions(res, allowed=args)
            if res not in found_solutions and verbose: 
                if to_matrix:
                    print(sol2matrix(res, dim=dim, use_alphabet=use_alphabet), end='\n\n')
                else:
                    print(res)
    
            found_solutions.add(res)
        
    if to_matrix:
        return [sol2matrix(solution, dim=dim, use_alphabet=use_alphabet) for solution in found_solutions]
    else: 
        return found_solutions


In [40]:
res = normalizers_v2(14, verbose=True)


14 point group: Group([ [ [ 0, 1 ], [ -1, -1 ] ], [ [ 0, -1 ], [ -1, 0 ] ] ])
group elements:
[ [ 1, 0 ], [ 0, 1 ] ]
[ [ -1, -1 ], [ 1, 0 ] ]
[ [ 0, 1 ], [ -1, -1 ] ]
[ [ 0, -1 ], [ -1, 0 ] ]
[ [ -1, 0 ], [ 1, 1 ] ]
[ [ 1, 1 ], [ 0, -1 ] ]

----------------normalizers-------------------------
(a_00 == 0, a_01 == x0, a_10 == x0, a_11 == 0)
(a_00 == x0, a_01 == x0, a_10 == 0, a_11 == -x0)
(a_00 == -x0, a_01 == 0, a_10 == x0, a_11 == x0)
(a_00 == x0, a_01 == 0, a_10 == 0, a_11 == x0)
(a_00 == x0, a_01 == x0, a_10 == -x0, a_11 == 0)
(a_00 == 0, a_01 == -x0, a_10 == x0, a_11 == x0)


In [42]:
el = res.pop()

In [34]:
_ = normalizers(14, verbose=True)


14 point group: Group([ [ [ 0, 1 ], [ -1, -1 ] ], [ [ 0, -1 ], [ -1, 0 ] ] ])
group elements:
[1 0]
[0 1]
[-1 -1]
[ 1  0]
[ 0  1]
[-1 -1]
[ 0 -1]
[-1  0]
[-1  0]
[ 1  1]
[ 1  1]
[ 0 -1]

----------------normalizers-------------------------
(x0 == x4, x3 == 0, x2 == 0, x1 == x4)
(x3 == 0, x2 == 0, x1 == 0, x0 == 0)
(x0 == x4, x3 == x4, x2 == -x4, x1 == 0)
(x1 == 0, x4 == -x2, x3 == x2, x0 == x2)
(x0 == 0, x3 == x4, x2 == x4, x1 == 0)
(x0 == x1, x4 == x1, x3 == 0, x2 == -x1)
(x0 == -x1, x4 == 0, x3 == x1, x2 == x1)


In [165]:
for i in range(1, 17):
    _ = normalizers(i, verbose=True)


1 point group: Group([],[ [ 1, 0 ], [ 0, 1 ] ])
group elements:
[1 0]
[0 1]

----------------normalizers-------------------------
False

2 point group: Group([ [ [ -1, 0 ], [ 0, -1 ] ] ])
group elements:
[1 0]
[0 1]
[-1  0]
[ 0 -1]

----------------normalizers-------------------------
False

3 point group: Group([ [ [ -1, 0 ], [ 0, 1 ] ] ])
group elements:
[1 0]
[0 1]
[-1  0]
[ 0  1]

----------------normalizers-------------------------
(a == x1, b == 0, c == 0, d == x0)

4 point group: Group([ [ [ -1, 0 ], [ 0, 1 ] ] ])
group elements:
[1 0]
[0 1]
[-1  0]
[ 0  1]

----------------normalizers-------------------------
(a == x1, b == 0, c == 0, d == x0)

5 point group: Group([ [ [ -1, 0 ], [ 0, 1 ] ] ])
group elements:
[1 0]
[0 1]
[-1  0]
[ 0  1]

----------------normalizers-------------------------
(a == x1, b == 0, c == 0, d == x0)

6 point group: Group([ [ [ -1, 0 ], [ 0, -1 ] ], [ [ -1, 0 ], [ 0, 1 ] ] ])
group elements:
[1 0]
[0 1]
[-1  0]
[ 0 -1]
[-1  0]
[ 0  1]
[ 1  0]
[ 0 -1]

-

In [162]:
_ = normalizers(17, verbose=True)


17 point group: Group([ [ [ 0, 1 ], [ -1, -1 ] ], [ [ -1, 0 ], [ 0, -1 ] ], 
  [ [ 0, -1 ], [ -1, 0 ] ] ])
group elements:
[1 0]
[0 1]
[-1 -1]
[ 1  0]
[ 0  1]
[-1 -1]
[-1  0]
[ 0 -1]
[ 1  1]
[-1  0]
[ 0 -1]
[ 1  1]
[ 1  0]
[-1 -1]
[-1 -1]
[ 0  1]
[0 1]
[1 0]
[-1  0]
[ 1  1]
[ 1  1]
[ 0 -1]
[ 0 -1]
[-1  0]

----------------normalizers-------------------------
(a == x0, b == 0, c == 0, d == x0)
(a == 0, b == 0, c == 0, d == 0)
(a == x0, b == x0, c == -x0, d == 0)
(a == 0, b == -x0, c == x0, d == x0)
(a == 1/2*x0, b == x0, c == -x0, d == -1/2*x0)
(a == 1/2*x0, b == -1/2*x0, c == 1/2*x0, d == x0)
(a == 2*x0, b == x0, c == -x0, d == x0)


KeyboardInterrupt: Ctrl-c pressed while running Gap

# Create self-similar action

In [37]:
G = gap(f"SpaceGroupIT(2, 13)")

In [38]:
gens = G.GeneratorsOfGroup()

In [39]:
gens = [matrix(QQ, el) for el in gens]
gens

[
[ 0  1  0]  [1 0 0]  [1 0 0]
[-1 -1  0]  [0 1 0]  [0 1 0]
[ 0  0  1], [1 0 1], [0 1 1]
]

In [25]:
T = matrix(QQ, [[1, -1, 0], [1, 2, 0], [0, 0, 1]])
T

[ 1 -1  0]
[ 1  2  0]
[ 0  0  1]

In [16]:
def self_similar(n, T, dim=2, verbose=False, gen_alphabet=False):
    G = gap(f'SpaceGroupIT({dim}, {n})')
    gens_G = G.GeneratorsOfGroup() 
    gens_G = [matrix(QQ, el) for el in gens_G]
    if verbose:
        print("=====================================================================")
    
    # check whether there exists virtual endomorphism
    conj_els = []
    for el in gens_G: 
        conj = T.inverse() * el * T 
        if verbose:
            print("\nconjugate el:")
            print(conj)
            print("conj in G:", conj in G)
        if conj not in G: 
            raise ValueError("Bad matrix T, there is no virtual endomorphism")
        conj_els.append(conj)
    
    # create subgroup as image of virtual endomorphism
    H = G.Subgroup(conj_els)
    if verbose: 
        print("----------------------------------------------------")
        print("Index of subgroup H:", G.Index(H))
    
    trans = G.RightTransversal(H)
    gens_H = [matrix(QQ, el) for el in H.GeneratorsOfGroup()]
    trans_els = [matrix(QQ, el) for el in trans.AsList()]
    trans_inv_els = [el.inverse() for el in trans_els]
    if verbose: 
        print("Transversal:")
        print(*trans_els, sep='\n\n')
    
    if gen_alphabet:
        names_G = {str(el): letter for el, letter in zip(gens_G, alphabet)}
    else: 
        names_G = {str(el): f"a_{i}" for i, el in enumerate(gens_G, 1)}
    
    # create self-similar action 
    res_map = {} 
    for a in gens_G: 
        for i, d_i in enumerate(trans_inv_els, 1):
            adi = a * d_i 
            d_j_index = trans.PositionCanonical(adi.inverse())
            d_j = trans_inv_els[int(d_j_index) - 1]
            
            if d_j.inverse() * a * d_i not in H: 
                print("Error, wrong coset")
                print(a, d_i, d_j)
                return -1
            
            tmp_res = T * (d_j.inverse() * a * d_i) * T.inverse() 
            res_map[(names_G[str(a)], i)] = (int(d_j_index), tmp_res)
    return res_map
    

In [17]:
self_similar(13, T, verbose=True, gen_alphabet=True)

NameError: name 'T' is not defined

In [65]:
T = matrix(QQ, [[0, -1, 0], [2, 0, 0], [0, 0, 1]])
self_similar(6, T, verbose=True, gen_alphabet=True)


conjugate el:
[-1  0  0]
[ 0 -1  0]
[ 0  0  1]
conj in G: True

conjugate el:
[ 1  0  0]
[ 0 -1  0]
[ 0  0  1]
conj in G: True

conjugate el:
[ 1  0  0]
[ 0  1  0]
[ 0 -1  1]
conj in G: True

conjugate el:
[1 0 0]
[0 1 0]
[2 0 1]
conj in G: True
----------------------------------------------------
Index of subgroup H: 2
Transversal:
[1 0 0]
[0 1 0]
[0 0 1]

[1 0 0]
[0 1 0]
[1 0 1]


{('a', 1): (
   [-1  0  0]
   [ 0 -1  0]
1, [ 0  0  1]
),
 ('a', 2): (
   [-1  0  0]
   [ 0 -1  0]
2, [ 0 -1  1]
),
 ('b', 1): (
   [ 1  0  0]
   [ 0 -1  0]
1, [ 0  0  1]
),
 ('b', 2): (
   [ 1  0  0]
   [ 0 -1  0]
2, [ 0 -1  1]
),
 ('c', 1): (
   [1 0 0]
   [0 1 0]
2, [0 1 1]
),
 ('c', 2): (
   [1 0 0]
   [0 1 0]
1, [0 0 1]
),
 ('d', 1): (
   [ 1  0  0]
   [ 0  1  0]
1, [-1  0  1]
),
 ('d', 2): (
   [ 1  0  0]
   [ 0  1  0]
2, [-1  0  1]
)}

In [75]:
T = matrix(QQ, [[1, -1, 0], [1, 1, 0], [0, 0, 1]])
self_similar(14, T, verbose=True, gen_alphabet=True)


conjugate el:
[-1  0  0]
[ 0 -1  0]
[ 0  0  1]
conj in G: True

conjugate el:
[ 0  1  0]
[-1  0  0]
[ 0  0  1]
conj in G: True

conjugate el:
[0 1 0]
[1 0 0]
[1 0 1]
conj in G: False


ValueError: Bad matrix T, there is no virtual endomorphism

In [68]:
G = gap.SpaceGroupIT(2, 8)
gens = [matrix(QQ, el) for el in G.GeneratorsOfGroup().AsList()]
gens

[
[-1  0  0]  [ -1   0   0]  [1 0 0]  [1 0 0]
[ 0 -1  0]  [  0   1   0]  [0 1 0]  [0 1 0]
[ 0  0  1], [1/2 1/2   1], [1 0 1], [0 1 1]
]

In [69]:
a, b, c, d = gens 

a * b 

[  1   0   0]
[  0  -1   0]
[1/2 1/2   1]

In [70]:
a * d

[-1  0  0]
[ 0 -1  0]
[ 0  1  1]

In [67]:
T = matrix(QQ, [[0, 2, 0], [1, 0, 0], [0, 0, 1]])
self_similar(9, T, verbose=True, gen_alphabet=True)


conjugate el:
[-1  0  0]
[ 0 -1  0]
[ 0  0  1]
conj in G: True

conjugate el:
[ 1  0  0]
[ 0 -1  0]
[ 0  0  1]
conj in G: True

conjugate el:
[  1   0   0]
[  0   1   0]
[1/2   1   1]
conj in G: False


ValueError: Bad matrix T, there is no virtual endomorphism

In [56]:
T = matrix(QQ, [[2, 2, 0], [-2, 0, 0], [0, 0, 1]])
self_similar(17, T, verbose=True, gen_alphabet=True)


conjugate el:
[ 0  1  0]
[-1 -1  0]
[ 0  0  1]
conj in G: True

conjugate el:
[-1  0  0]
[ 0 -1  0]
[ 0  0  1]
conj in G: True

conjugate el:
[ 1  1  0]
[ 0 -1  0]
[ 0  0  1]
conj in G: True

conjugate el:
[1 0 0]
[0 1 0]
[2 2 1]
conj in G: True

conjugate el:
[ 1  0  0]
[ 0  1  0]
[-2  0  1]
conj in G: True
----------------------------------------------------
Index of subgroup H: 4
Transversal:
[1 0 0]
[0 1 0]
[0 0 1]

[1 0 0]
[0 1 0]
[1 0 1]

[1 0 0]
[0 1 0]
[0 1 1]

[1 0 0]
[0 1 0]
[1 1 1]


{('a', 1): (
   [ 0  1  0]
   [-1 -1  0]
1, [ 0  0  1]
),
 ('a', 2): (
   [ 0  1  0]
   [-1 -1  0]
4, [ 0  1  1]
),
 ('a', 3): (
   [ 0  1  0]
   [-1 -1  0]
2, [ 0  0  1]
),
 ('a', 4): (
   [ 0  1  0]
   [-1 -1  0]
3, [-1  0  1]
),
 ('b', 1): (
   [-1  0  0]
   [ 0 -1  0]
1, [ 0  0  1]
),
 ('b', 2): (
   [-1  0  0]
   [ 0 -1  0]
2, [ 0  1  1]
),
 ('b', 3): (
   [-1  0  0]
   [ 0 -1  0]
3, [-1 -1  1]
),
 ('b', 4): (
   [-1  0  0]
   [ 0 -1  0]
4, [-1  0  1]
),
 ('c', 1): (
   [-1  0  0]
   [ 1  1  0]
1, [ 0  0  1]
),
 ('c', 2): (
   [-1  0  0]
   [ 1  1  0]
3, [ 0  1  1]
),
 ('c', 3): (
   [-1  0  0]
   [ 1  1  0]
2, [-1 -1  1]
),
 ('c', 4): (
   [-1  0  0]
   [ 1  1  0]
4, [-1  0  1]
),
 ('d', 1): (
   [ 1  0  0]
   [ 0  1  0]
2, [ 0 -1  1]
),
 ('d', 2): (
   [1 0 0]
   [0 1 0]
1, [0 0 1]
),
 ('d', 3): (
   [ 1  0  0]
   [ 0  1  0]
4, [ 0 -1  1]
),
 ('d', 4): (
   [1 0 0]
   [0 1 0]
3, [0 0 1]
),
 ('e', 1): (
   [1 0 0]
   [0 1 0]
3, [1 1 1]
),
 ('e', 2): (
   [1 0 0]
   [0 1 0]
4, [1 

In [31]:
self_similar(7, T, verbose=True, gen_alphabet=True)


conjugate el:
[-1  0  0]
[ 0 -1  0]
[ 0  0  1]
conj in G: True

conjugate el:
[  1   0   0]
[  0  -1   0]
[  0 3/2   1]
conj in G: False


ValueError: Bad matrix T, there is no virtual endomorphism

In [45]:
var('a b c d')
A = matrix([[0, a], [a, 0]])
A.charpoly()

x^2 - a^2

In [49]:
A - x

[-x  a]
[ a -x]

In [50]:
G = gap.SpaceGroupIT(2, 13)
G.PointGroup()

Group([ [ [ 0, 1 ], [ -1, -1 ] ] ])